In [63]:
from __future__ import annotations
from typing import TYPE_CHECKING
import polars as pl

from common.jupyter import get_nodes, plot_node, get_datasets
from common import polars as ppl
from rich import print
import scipy
if TYPE_CHECKING:
    from nodes.context import Context
    from nodes.node import Node
    from nodes.actions.shift import ShiftParameterValue
    from nodes.metric import DimensionalMetric, DimensionalFlow


In [64]:
from typing import cast
from nodes.actions.action import ActionNode
from nodes.constants import YEAR_COLUMN
from nodes.datasets import DVCDataset

nodes = get_nodes('zuerich')
ctx: Context = nodes.context
act_nodes = cast(list[ActionNode], [
    nodes['fossil_fuel_heater_to_district_heat'],
    nodes['fossil_fuel_heater_to_heat_pumps'],
    nodes['fossil_fuel_heater_to_other'],
])
outcome_node: Node = nodes['building_heat_mix']
ds = DVCDataset('zuerich/building_heat_mix_goals', tags=[])
df = ds.get_copy(ctx)
outcome_cols = outcome_node.compute().paths.to_wide(only_category_names=True).metric_cols
goal_df = df.paths.to_wide(only_category_names=True)
# ensure the columns are in the same order
goal_df = goal_df.select(outcome_cols + [YEAR_COLUMN])


2023-11-08 01:49:21.296354 DEBUG      [3ZT327F0PG] Cache initialized                                   ]8;id=122455;file:///home/jey/sync/devel/kausal-paths/common/cache.py#99\common.cache]8;;\

In [66]:
from functools import partial
from typing import Callable
import networkx as nx
from scipy import optimize
import yaml

from nodes.constants import FORECAST_COLUMN
if TYPE_CHECKING:
    from nodes.actions.shift import ShiftAmount


def compute_and_compare(vals: list[float], goal: pl.DataFrame, value_setters: list[Callable]):
    # set parameters
    for val, set_value in zip(vals, value_setters):
        set_value(val)
    df = outcome_node.compute().paths.to_wide(only_category_names=True).drop(FORECAST_COLUMN)
    outcome = df.filter(pl.col('Year') == goal['Year'][0]).drop('Year')
    diff = outcome - goal
    m = [val ** 2 for val in diff.to_numpy()[0]]
    return m


def run(start_year: int, target_year: int):
    target_year_goal = goal_df.filter(pl.col('Year') == target_year)

    # Disable output caching for all nodes in 
    path_nodes = set()

    x0 = []
    value_setters = []
    lower_bounds = []
    upper_bounds = []
    diff_steps = []

    def set_source_value(start: ShiftAmount | None, end: ShiftAmount, new_val: float):
        if start is not None:
            start.source_amount = new_val
        end.source_amount = new_val

    def set_dest_value(start: ShiftAmount | None, end: ShiftAmount, idx: int, new_val: float):
        if start is not None:
            start.dest_amounts[idx] = new_val
        end.dest_amounts[idx] = new_val

    for act in act_nodes:
        all_paths = list(nx.all_simple_paths(ctx.node_graph, source=act.id, target=outcome_node.id))
        assert len(all_paths)
        for path in all_paths:
            path_nodes.update(path)

        param = act.get_parameter('shift')
        val: ShiftParameterValue = param.value.model_copy(deep=True)
        param.value = val

        # remove all the values after our start year
        for entry in val.root:
            entry.amounts = list(sorted([a for a in entry.amounts if a.year <= start_year], key=lambda x: x.year))
            start = entry.amounts[-1]
            if start.year != start_year:
                start = entry.amounts[-1].model_copy()
                start.year = start_year
                entry.amounts.append(start)
            end = start.model_copy(update=dict(year=target_year, deep=True))
            entry.amounts.append(end)

            x0.append(start.source_amount)
            value_setters.append(partial(set_source_value, start, end))
            if start.source_amount < 0:
                lower_bounds.append(-100)
                upper_bounds.append(0)
            elif start.source_amount > 0:
                lower_bounds.append(0)
                upper_bounds.append(100)

            diff_steps.append(0.01)
            if len(start.dest_amounts) == 1:
                continue
            sum_amounts = sum(start.dest_amounts)
            for idx, amount in enumerate(start.dest_amounts):
                x0.append(amount)
                value_setters.append(partial(set_dest_value, start, end, idx))
                lower_bounds.append(0)
                upper_bounds.append(sum_amounts)
                diff_steps.append(0.1 * sum_amounts)

    for node_id in path_nodes:
        ctx.get_node(node_id).disable_cache = True

    df = outcome_node.compute().paths.to_wide(only_category_names=True)\
        .drop(FORECAST_COLUMN).filter(pl.col(YEAR_COLUMN) == target_year)\
        .drop(YEAR_COLUMN)

    print('start:\n%s\nlb:\n%s\nub:\n%s\n' % (x0, lower_bounds, upper_bounds))
    print(target_year_goal)
    with ctx.run():
        res = optimize.least_squares(
            compute_and_compare,
            x0, bounds=[lower_bounds, upper_bounds],
            max_nfev=500,
            kwargs=dict(
                goal=target_year_goal,
                value_setters=value_setters,
            )
        )
        #df = outcome_node.compute().paths.to_wide(only_category_names=True)\
        #    .drop(FORECAST_COLUMN).filter(pl.col(YEAR_COLUMN) == target_year)\
        #    .drop(YEAR_COLUMN)
        #print(df)
        #print(x0)
        print([round(x, 2) for x in res.x])
        for x, set_value in zip(res.x, value_setters):
            set_value(round(float(x), 2))

orig_values = []
for act in act_nodes:
    orig_values.append(act.get_parameter_value('shift'))

last_hist_year: int = list(outcome_node.get_output_pl().filter(~pl.col(FORECAST_COLUMN))[YEAR_COLUMN])[-1]
years = sorted(goal_df[YEAR_COLUMN])
start_year = last_hist_year + 1
try:
    for year in years:
        print(year)
        run(start_year, year)
        start_year = year + 1
finally:
    df = outcome_node.compute().paths.to_wide(only_category_names=True).filter(pl.col[FORECAST_COLUMN])

    for act, orig_val in zip(act_nodes, orig_values):
        val: ShiftParameterValue = act.get_parameter_value('shift')
        act.set_parameter_value('shift', orig_val)
        orig_data = orig_val.model_dump(exclude_none=False)
        new_data = val.model_dump(exclude_none=False)
        print('- id: %s\n%s\n->\n%s\n\n' % (act.id, yaml.safe_dump(orig_data), yaml.safe_dump(new_data)))

#ctx.node_graph

2025

start:
[-1.1258676411000659, -1.1287019465982544, -1.0161430178604294, -1.1226970661949631, -0.07673651912593314, 
10.90564858176264, 6.99837833118664, -0.10713872448913278, 10.819390496002105, 3.273168036639447]
lb:
[-100, -100, -100, -100, -100, 0, 0, -100, 0, 0]
ub:
[0, 0, 0, 0, 0, 17.904026912949277, 17.904026912949277, 0, 14.092558532641553, 14.092558532641553]

shape: (1, 7)
┌──────────────┬──────────────┬──────────────┬──────────┬──────────────┬──────────────┬────────────┐
│ [%] fuel_oil ┆ [%]          ┆ [%] district ┆ [%] wood ┆ [%] solar_co ┆ [%]          ┆ [idx] Year │
│ ---          ┆ natural_gas  ┆ _heat        ┆ ---      ┆ llectors     ┆ heat_pumps   ┆ ---        │
│ f64          ┆ ---          ┆ ---          ┆ f64      ┆ ---          ┆ ---          ┆ i64        │
│              ┆ f64          ┆ f64          ┆          ┆ f64          ┆ f64          ┆            │
╞══════════════╪══════════════╪══════════════╪══════════╪══════════════╪══════════════╪════════════╡
│ 15.6         ┆ 46.0         ┆ 24.0         ┆ 5.0      ┆ 0.4          ┆ 9.0          ┆ 2025       │
└──────────────┴──────────────┴──────────────┴──────────┴──────────────┴──────────────┴────────────┘

2023-11-08 01:54:33.431909 DEBUG      [3ZT327F0PG] Start execution run                                 ]8;id=664930;file:///home/jey/sync/devel/kausal-paths/common/cache.py#116\common.cache]8;;\

[-0.51, -1.27, -0.59, -1.14, -0.05, 16.11, 11.88, -0.02, 2.5, 11.61]

...

shape: (1, 7)
┌──────────────┬──────────────┬──────────────┬──────────┬──────────────┬──────────────┬────────────┐
│ [%] fuel_oil ┆ [%]          ┆ [%] district ┆ [%] wood ┆ [%] solar_co ┆ [%]          ┆ [idx] Year │
│ ---          ┆ natural_gas  ┆ _heat        ┆ ---      ┆ llectors     ┆ heat_pumps   ┆ ---        │
│ f64          ┆ ---          ┆ ---          ┆ f64      ┆ ---          ┆ ---          ┆ i64        │
│              ┆ f64          ┆ f64          ┆          ┆ f64          ┆ f64          ┆            │
╞══════════════╪══════════════╪══════════════╪══════════╪══════════════╪══════════════╪════════════╡
│ 0.0          ┆ 9.0          ┆ 49.0         ┆ 6.0      ┆ 1.0          ┆ 35.0         ┆ 2040       │
└──────────────┴──────────────┴──────────────┴──────────┴──────────────┴──────────────┴────────────┘

2023-11-08 01:54:54.199880 DEBUG      [3ZT327F0PG] Start execution run                                 ]8;id=956615;file:///home/jey/sync/devel/kausal-paths/common/cache.py#116\common.cache]8;;\

[-0.51, -1.21, -0.54, -1.15, -0.13, 17.1, 7.95, -0.01, 4.55, 11.52]

2023-11-08 01:55:00.601504 DEBUG      [3ZT327F0PG] End execution run (computation 6401.60 ms, 257      ]8;id=728662;file:///home/jey/sync/devel/kausal-paths/common/cache.py#149\common.cache]8;;\
                                    reqs; caching 0 new objects took 0.00 ms)                                      

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:129                                                                                  │
│                                                                                                  │
│   126 finally:                                                                                   │
│   127 │   for act, orig_val in zip(act_nodes, orig_values):                                      │
│   128 │   │   val: ShiftParameterValue = act.get_parameter_value('shift')                        │
│ ❱ 129 │   │   act.set_parameter_value('shift', orig_val)                                         │
│   130 │   │   orig_data = orig_val.model_dump(exclude_none=False)                                │
│   131 │   │   new_data = val.model_dump(exclude_none=False)                                      │
│   132 │   │   print('- id: %s\n%s\n->\n%s\n\n' % (act.id, yaml.safe_dump(orig_data), yaml.safe   │
│                                                                                                  │
│ /home/jey/sync/devel/kausal-paths/nodes/node.py:488 in set_parameter_value                       │
│                                                                                                  │
│    485 │   │   #     self.parameters[local_id] = Parameter()                                     │
│    486 │   │   if local_id not in self.parameters:                                               │
│    487 │   │   │   raise NodeError(self, f"Local parameter {local_id} not found for node {self.  │
│ ❱  488 │   │   self.parameters[local_id].set(value)                                              │
│    489 │                                                                                         │
│    490 │   def get_input_datasets_pl(self, tag: str | None = None, exclude_tags: list[str] | No  │
│    491 │   │   dfs: List[ppl.PathsDataFrame] = []                                                │
│                                                                                                  │
│ /home/jey/sync/devel/kausal-paths/params/param.py:77 in set                                      │
│                                                                                                  │
│    74 │                                                                                          │
│    75 │   def set(self, value: V, notify: bool = True):                                          │
│    76 │   │   prev_val = getattr(self, 'value', None)                                            │
│ ❱  77 │   │   self.value = self.clean(value)                                                     │
│    78 │   │   if notify and not self.is_value_equal(prev_val):                                   │
│    79 │   │   │   self.notify_change()                                                           │
│    80                                                                                            │
│                                                                                                  │
│ /home/jey/sync/devel/kausal-paths/nodes/actions/shift.py:113 in clean                            │
│                                                                                                  │
│   110 │                                                                                          │
│   111 │   def clean(self, value: Any) -> ShiftParameterValue:                                    │
│   112 │   │   if not isinstance(value, list):                                                    │
│ ❱ 113 │   │   │   raise ValidationError(self, "Input must be a list")                            │
│   114 │   │                                                                                      │
│   115 │   │   try:                                                                               │
│   116 │   │   │   return ShiftParameterValue.model_validate